# Convert the data for GAMA

### make the training json for GAMA

### make training data, with key "input"

In [1]:
import pandas as pd
import json

# Load the CSV file
df = pd.read_csv("train_mentalhealth.csv")  # change sep if your file is comma-separated

# Function to construct the desired format
def row_to_dict(row):
    return {
        "audio_id": f"/data/amey_2311cs10/debayan/train_mentalhealth/{row['file_id']}.wav",
        "instruction": row["Question"],
        "input":"",
        "dataset": "",
        "task": "",
        "output": f"<Answer>{row['Answer']}</Answer><Reasoning>{row['Reasoning']}</Reasoning>"
        
    }

# Apply transformation
data = [row_to_dict(row) for _, row in df.iterrows()]

# Save to JSON
with open("train_json_for_gama1.json", "w") as f:
    json.dump(data, f, indent=2)

print("DONE")

DONE


### for GAMA, but not train only Inference

In [10]:
import pandas as pd
import json

# Load the CSV
df = pd.read_csv("test_mentalhealth.csv")  # replace with your CSV filename

json_data = []

for _, row in df.iterrows():
    entry = {
        "audio_id": f"/data/amey_2311cs10/AudioLM_baselines/GAMA/test_mentalhealth_16kHz/{row['file_id']}.wav",
        "input": "",
        "dataset": "",
        "task": "open-ended question",
        "instruction_output": [
            {
                "instruction": row["Question"],
                "output": f"<Answer>{row['Answer']}</Answer><Reasoning>{row['Reasoning']}</Reasoning>"
            }
        ],
        "timestamp_events": "",
        "caption": ""
    }
    json_data.append(entry)

# Save to a JSON file
with open("test_json_for_gama.json", "w") as f:
    json.dump(json_data, f, indent=4)

In [ ]:
# !cp debayan/test_json_for_gama.json AudioLM_baselines/GAMA/

In [ ]:
AudioLM_baselines/GAMA/test_mentalhealth_16khz/mfe8OzzArGc_chunk29_data1_task3.wav

_____________________________________________________________________________________________________________

# Convert the data for Audio-flamingo2

In [2]:
import pandas as pd
import json

### Training json

### Train json (with .wav extension)

In [3]:
# Load CSV
csv_path = 'train_mentalhealth.csv'  # replace with your actual CSV file path
df = pd.read_csv(csv_path)

# Construct JSON
json_data = {
    "split": "train",
    "split_path": "train_mentalhealth",
    "flamingo_task": "audio-qa-reasoning",
    "total_num": len(df),
    "data": {}
}

# Populate data
for idx, row in df.iterrows():
    json_data["data"][str(idx)] = {
        "name": row["file_id"] + ".wav",
        "prompt": row["Question"],
        "output": f"<Answer> {row['Answer']} </Answer> <Reasoning> {row['Reasoning']} </Reasoning>",
        "duration": 30
    }

# Save to JSON
output_path = 'audioflamingo2_train_ext.json'  # name of output file
with open(output_path, 'w') as f:
    json.dump(json_data, f, indent=4)

### Test json

### Test json (with .wav extension)

In [ ]:
# Load CSV
csv_path = 'test_mentalhealth.csv'  # replace with your actual CSV file path
df = pd.read_csv(csv_path)

# Construct JSON
json_data = {
    "split": "test",
    "split_path": "test_mentalhealth",
    "flamingo_task": "audio-qa-reasoning",
    "total_num": len(df),
    "data": {}
}

# Populate data
for idx, row in df.iterrows():
    json_data["data"][str(idx)] = {
        "name": row["file_id"] + ".wav",
        "prompt": row["Question"],
        "output": f"<Answer> {row['Answer']} </Answer> <Reasoning> {row['Reasoning']} </Reasoning>",
        "duration": 30
    }

# Save to JSON
output_path = 'audioflamingo2_test_ext.json'  # name of output file
with open(output_path, 'w') as f:
    json.dump(json_data, f, indent=4)

_______________________________________________________________________

# Convert the data to Qwen 2 Audio

In [1]:
import pandas as pd
import json

In [2]:
csv_path = 'test_mentalhealth.csv'  # replace with your actual CSV file path
df = pd.read_csv(csv_path)

In [4]:
# Prepare conversations list
conversations = []

for _, row in df.iterrows():
    conv = [
        {
            "role": "system",
            "content": "Output should be in <Answer>...</Answer><Reasoning>...</Reasoning> format."
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "audio",
                    "audio_url": f"/data/amey_2311cs10/AudioLM_baselines/GAMA/test_mentalhealth_16kHz/{row['file_id']}.wav"
                },
                {
                    "type": "text",
                    "text": row["Question"]
                }
            ]
        }
    ]
    conversations.append({"conversation": conv})

# Save to JSON file
with open("qwen2audio_inference.json", "w") as f:
    json.dump(conversations, f, indent=4)

____________________________________________________

# Convert the data for Salmonn

In [2]:
import pandas as pd
import json

In [3]:
csv_path = 'test_mentalhealth.csv'  # replace with your actual CSV file path
df = pd.read_csv(csv_path)

In [11]:
# unique_tasks = df['Question'].unique()

# # Assuming unique_tasks is a NumPy array or list-like
# task_dict = {task: task for task in unique_tasks}

# # To print it as a JSON-formatted string
# # print(json.dumps(task_dict, indent=4, ensure_ascii=False))

# with open('test_prompt_salmonn.json', 'w', encoding='utf-8') as f:
#     json.dump(task_dict, f, indent=4, ensure_ascii=False)

In [8]:
# Build the list of annotations
annotations = []
for _, row in df.iterrows():
    annotation = {
        "path": f"./test_mentalhealth_16kHz/{row['file_id']}.wav",
        "text": f"<Answer> {row['Answer']} </Answer> <Reasoning> {row['Reasoning']} </Reasoning>",
        "task": "QA",
        "Q": row['Question']
    }
    annotations.append(annotation)

# Final dictionary
final_json = {
    "annotation": annotations
}

# Save to JSON file
with open('data_test_salmonn_annotations.json', 'w', encoding='utf-8') as f:
    json.dump(final_json, f, indent=4, ensure_ascii=False)

## Convert the data for Salmonn (Few shot prompting for tasks) to get reasoning

In [10]:
import pandas as pd
import json
import re

In [11]:
csv_path = 'test_mentalhealth.csv'  # replace with your actual CSV file path
df = pd.read_csv(csv_path)

In [14]:
task_examples = {
    "task1": """Below is an example of the expected output format for this task.\nOutput: <s> <Answer> Yes, the patient suffers from stress. </Answer> <Reasoning> The patient openly discusses their history of anxiety and how it used to only affect them during specific events. However, they now express that their anxiety has escalated to a new level and is impacting their day-to-day life. This indicates that the patient is experiencing increased stress and anxiety symptoms, suggesting they are suffering from ongoing stress. </Reasoning> </s>""",
    
    "task2": """Below is an example of the expected output format for this task.\nOutput: <s> <Answer> This shows suicide or self-harm tendency. </Answer> <Reasoning> The use of phrases like ""I hate everything and everybody"" and ""This life is not worth living"" indicate a strong sense of hopelessness and despair. The mention of love not being like the movies and the phrase ""we're fucked"" further suggests a negative perspective on life and relationships. The use of profanity and the intensity of the emotions expressed suggest a high level of distress and potential risk for self-harm or suicide. </Reasoning> </s>""",
    
    "task3": """Below is an example of the expected output format for this task.\nOutput: <s> <Answer> This patient shows the stress cause related to everyday decision making. </Answer> <Reasoning> The patient mentions having to end a conversation because of needing to finish work at Starbucks. This suggests that the patient is experiencing stress due to having to make a decision between spending time talking and completing work. This falls under the category of everyday decision making as a stress cause. </Reasoning> </s>""",

    "task4": """Below is an example of the expected output format for this task.\nOutput: <s> <Answer> Yes, this wellness dimension exists here. </Answer> <Reasoning> The wellness dimension of emotional aspect exists in the patient. The individual is expressing feelings of self-blame, regret, and a desire for death. This demonstrates awareness of their own emotions and a lack of acceptance of their current state. </Reasoning> </s>"""
}

In [15]:
annotations = []

for _, row in df.iterrows():
    match = re.search(r'task\d+', row['file_id'])
    task_key = match.group() if match else 'task1'
    
    task_instruction = task_examples.get(task_key, "")
    full_question = f"{row['Question']}\n\n{task_instruction}"
    
    annotation = {
        "path": f"./test_mentalhealth_16kHz/{row['file_id']}.wav",
        "text": f"<Answer> {row['Answer']} </Answer> <Reasoning> {row['Reasoning']} </Reasoning>",
        "task": "QA",
        "Q": full_question
    }
    annotations.append(annotation)

final_json = {
    "annotation": annotations
}

# Save to JSON file
with open('test_mentalhealth_salmonn.json', 'w', encoding='utf-8') as f:
    json.dump(final_json, f, indent=4, ensure_ascii=False)

_______________________________________________________
# Convert data for LTU & LTU-AS

In [2]:
import pandas as pd
import json

In [8]:
df = pd.read_csv("test_audmind.csv")

# Prepare the list of JSON entries
json_data = []

for _, row in df.iterrows():
    entry = {
        "instruction": row["Question"],
        "input": row["text"],
        "audio_id": f"/data/amey_2311cs10/debayan/test_mentalhealth_16kHz/{row['file_id']}.wav",
        "dataset": "",
        "task": "open-ended question",
        "output": f"<Answer>{row['Answer']}</Answer> <Reasoning>{row['Reasoning']}</Reasoning>"
    }
    json_data.append(entry)

# Save as JSON
with open("ltu_test_audmind.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False)

print("JSON saved as ltu_test_audmind.json")

JSON saved as ltu_test_audmind.json


______________________________________________________
# Convert data for Mellow

In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv("test_audmind.csv")

# Prepare the list of JSON entries
json_data = []

for _, row in df.iterrows():
    entry = {
        
        "taskname": "audmind",
        "filepath1": f"/data/amey_2311cs10/debayan/test_mentalhealth_16kHz/{row['file_id']}.wav",
        "filepath2": "",
        "caption1": row["text"],
        "caption2": "",
        "input": row['Question'],
        "answer": f"<Answer>{row['Answer']}</Answer> <Reasoning>{row['Reasoning']}</Reasoning>",
        "subtype": "mellow_test_audmind.json"
    }
    json_data.append(entry)

# Save as JSON
with open("mellow_test_audmind.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False)

print("JSON saved as mellow_test_audmind.json")

JSON saved as mellow_test_audmind.json
